In [1]:
#load modules
import numpy as np
import matplotlib.pyplot as plt
import h5py
import avulHPC
import rasterio

In [2]:
#avulsions that have been both found by the method and previously found are at index locations 3 and 4 in this list
PrvFndAvLong = [-66.1590,-66.3622,-66.3509,-65.8537,-65.8378,-65.4020,-65.2262,-65.2152,-65.3422,-65.3340,-65.3038,-64.8657,-64.6130]
PrvFndAvLat  = [-16.0657,-15.8689,-15.7946,-15.8940,-16.3224,-16.8264,-16.4686,-16.2683,-16.9395,-16.9539,-17.0025,-17.2099,-17.3135]

In [3]:
#load mask. Only need to run this cell on images that haven't been re-georeferenced using the rasterio.open command in one of the cells below
mask = plt.imread('WholeTestAreaDeepwaterPlusManualMask.png')
#mask = plt.imread('WholeTestAreaTempCenterlinesThreshold1.png')
mask[mask == 255] = 1

In [12]:
mask = mask.astype('uint8') 

In [13]:
#make it so that you can get the lat long of each pixel in the mask
#Do this by getting transform for image
left = -68.91852401882868
bottom = -17.576032607204688 
right = -64.50312473432442
top = -13.160633322700418
MaskTran = rasterio.transform.from_bounds(left, bottom, right, top, 16384, 16384)

In [14]:
# re-georeference the mask using the surface water dataset cooridante reference system (epsg 4326)
RasterMask = rasterio.open('./WholeTestAreaDeepwaterPlusManualMask.png', 'w', driver='PNG',
                            height = 16384, width = 16384,
                            count=1, dtype = 'uint8',
                            crs=rasterio.crs.CRS.from_epsg(4326),
                            transform=MaskTran)

In [15]:
RasterMask.write(mask,1)

In [16]:
RasterMask.close()

In [18]:
RasterMask = rasterio.open('./WholeTestAreaDeepwaterPlusManualMask.png', 'r')

In [33]:
#now go pixel by pixel and if it is a white pixel see if it matches up with the previously found avulsions (with a tolerance factor)
tol = 80 #tol is the number of pixels you will look around the pixel to see if water is present. Make this the equivalent of about .02 degrees. 80x80 pixels pretty good 
IsAvulMasked = np.zeros(13)
for x in range(13):
    long = PrvFndAvLong[x]
    lat = PrvFndAvLat[x]
    py, px = RasterMask.index(long,lat)
    #now search in a tol x tol sized window around the location
    # Build an NxN window
    window = rasterio.windows.Window(px - tol//2, py - tol//2, tol, tol)
    print(window)

    # Read the data in the window
    # clip is a nbands * N * N numpy array
    clip = RasterMask.read(window=window)
    #want at least 1% of window area to contain data
    if np.sum(clip)>64:
        IsAvulMasked[x] = 0
    else:
        IsAvulMasked[x] = 1

Window(col_off=10199, row_off=10739, width=80, height=80)
Window(col_off=9445, row_off=10009, width=80, height=80)
Window(col_off=9487, row_off=9733, width=80, height=80)
Window(col_off=11332, row_off=10102, width=80, height=80)
Window(col_off=11391, row_off=11692, width=80, height=80)
Window(col_off=13008, row_off=13562, width=80, height=80)
Window(col_off=13660, row_off=12234, width=80, height=80)
Window(col_off=13701, row_off=11491, width=80, height=80)
Window(col_off=13230, row_off=13982, width=80, height=80)
Window(col_off=13260, row_off=14035, width=80, height=80)
Window(col_off=13372, row_off=14215, width=80, height=80)
Window(col_off=14998, row_off=14985, width=80, height=80)
Window(col_off=15936, row_off=15369, width=80, height=80)


In [10]:
pixloc = RasterMask.index(long,lat)

In [22]:
mask = RasterMask.read(1)

In [34]:
IsAvulMasked

array([1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1.])